In [5]:
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, BatchNormalization, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
import tifffile
import matplotlib.pyplot as plt

In [7]:

# Function to split an image into tiles
def split_image_into_tiles(image_path, mask_path, tile_size):
    img = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)
    mask = mask[:, :, 0] if len(mask.shape) == 3 else mask
    print(f"Original Image Dimensions: {img.shape}")
    tiles_img = []
    tiles_mask = []

    for x in range(0, img.shape[1], tile_size):
        for y in range(0, img.shape[0], tile_size):
            tile_img = img[y:y+tile_size, x:x+tile_size, :] if len(img.shape) == 3 else img[y:y+tile_size, x:x+tile_size]
            tile_mask = mask[y:y+tile_size, x:x+tile_size]

            # Resize to the desired size
            tile_img = cv2.resize(tile_img, (size, size))
            tile_mask = cv2.resize(tile_mask, (size, size))
            tile_mask = (tile_mask > 0).astype(np.uint8)

            tiles_img.append(tile_img)
            tiles_mask.append(tile_mask)

    # Calculate the number of tiles needed to form a perfect square
    num_tiles = len(tiles_img)
    perfect_square_size = int(np.ceil(np.sqrt(num_tiles)))
    total_tiles_needed = perfect_square_size**2
    num_tiles_to_add = total_tiles_needed - num_tiles

    # Pad the list of tiles with zeros
    for _ in range(num_tiles_to_add):
        tiles_img.append(np.zeros((size, size, img.shape[2]), dtype=np.uint8))
        tiles_mask.append(np.zeros((size, size), dtype=np.uint8))

    tiles_img = np.array(tiles_img)
    tiles_mask = np.array(tiles_mask)

    return tiles_img, tiles_mask

# Function to load data
def load_data(image_dir, mask_dir, tile_size):
    images = []
    masks = []

    # Sort filenames to ensure consistency
    image_filenames = sorted(os.listdir(image_dir))
    mask_filenames = sorted(os.listdir(mask_dir))

    for image_filename in image_filenames:
        if image_filename.endswith(".TIF"):
            mask_filename = image_filename.replace(".TIF", "_mask.TIF")

            if mask_filename in mask_filenames:
                image_path = os.path.join(image_dir, image_filename)
                mask_path = os.path.join(mask_dir, mask_filename)

                print(f"Processing Image: {image_filename}, Mask: {mask_filename}")

                img, mask = split_image_into_tiles(image_path, mask_path, tile_size)

                images.extend(img)
                masks.extend(mask)

    return np.array(images), np.array(masks)


In [8]:

import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import (Input, Conv2D, Conv2DTranspose,
                                     Concatenate, BatchNormalization,
                                     Activation, Lambda)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (ModelCheckpoint, EarlyStopping,
                                        LearningRateScheduler)
from sklearn.model_selection import train_test_split

# ------------------------------------------------------------------
# 1. Data‑loading stub (⇢ REPLACE with real loader)
# ------------------------------------------------------------------



# ------------------------------------------------------------------
# 2. ResNet50‑U‑Net construction
# ------------------------------------------------------------------
def resnet50_unet(input_size=(256, 256, 3),
                  freeze_encoder=True,
                  decoder_filters=(512, 256, 128, 64),
                  upsample_mode="transpose",
                  final_activation="sigmoid"):
    """Return a compiled ResNet50‑U‑Net Keras model."""
    # --- Encoder (ResNet50, ImageNet) ------------------------------
    resnet = ResNet50(weights="imagenet",
                      include_top=False,
                      input_tensor=Input(shape=input_size))

    # Feature maps for skip connections
    skip_names = [
        "conv1_relu",          # 128×128×64
        "conv2_block3_out",    # 64×64×256
        "conv3_block4_out",    # 32×32×512
        "conv4_block6_out"     # 16×16×1024
    ]
    encoder_output = resnet.get_layer("conv5_block3_out").output  # 8×8×2048

    if freeze_encoder:
        for layer in resnet.layers:
            layer.trainable = False

    # --- Decoder helper -------------------------------------------
    def upsample(x, filters, name):
        if upsample_mode == "transpose":
            return Conv2DTranspose(filters, 2, strides=2, padding="same",
                                   name=f"{name}_up")(x)
        # bilinear resize + conv (optional)
        x = Lambda(lambda t: tf.image.resize(t,
                                             (t.shape[1] * 2, t.shape[2] * 2),
                                             method="bilinear"),
                   name=f"{name}_resize")(x)
        return Conv2D(filters, 3, padding="same", name=f"{name}_conv")(x)

    # --- Decoder --------------------------------------------------
    x = encoder_output
    for i, (skip_name, f) in enumerate(zip(reversed(skip_names), decoder_filters), 1):
        x = upsample(x, f, name=f"dec{i}")
        skip = resnet.get_layer(skip_name).output
        x = Concatenate(name=f"concat{i}")([x, skip])

        x = Conv2D(f, 3, padding="same", name=f"dec{i}_conv1")(x)
        x = BatchNormalization(name=f"dec{i}_bn1")(x)
        x = Activation("relu", name=f"dec{i}_act1")(x)

        x = Conv2D(f, 3, padding="same", name=f"dec{i}_conv2")(x)
        x = BatchNormalization(name=f"dec{i}_bn2")(x)
        x = Activation("relu", name=f"dec{i}_act2")(x)

    # Final upsample to 256×256
    x = upsample(x, decoder_filters[-1] // 2, name="dec_final")

    # Output mask
    mask = Conv2D(1, 1, activation=final_activation,
                  padding="same", name="mask")(x)

    # Safeguard resize (identity for 256×256)
    mask = Lambda(lambda t: tf.image.resize(t,
                                            (input_size[0], input_size[1]),
                                            method="bilinear"),
                  name="identity_resize")(mask)

    model = Model(inputs=resnet.input, outputs=mask, name="ResNet50_UNet")
    model.compile(optimizer="adam", loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model


# ------------------------------------------------------------------
# 3. Paths and hyper‑parameters
# ------------------------------------------------------------------
IMAGE_DIR = "datasets2/images"
MASK_DIR  = "datasets2/masks"
TILE_SIZE = 256
size = 256
BATCH_SIZE = 32
EPOCHS = 100

# ------------------------------------------------------------------
# 4. Prepare data
# ------------------------------------------------------------------
X, y = load_data(IMAGE_DIR, MASK_DIR, TILE_SIZE)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42)

# Add channel dimension to masks
y_train = y_train[..., np.newaxis]
y_val   = y_val[..., np.newaxis]
y_test  = y_test[..., np.newaxis]

# Data augmentation (same settings as VGG16 pipeline)
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8, 1.2]
)

# ------------------------------------------------------------------
# 5. Callbacks
# ------------------------------------------------------------------
def lr_schedule(epoch):
    base_lr = 1e-4
    decay   = 0.9
    return base_lr * (decay ** (epoch // 10))

lr_scheduler = LearningRateScheduler(lr_schedule)

checkpointer  = ModelCheckpoint("best_resnet_unet_1572025.h5",
                                monitor="val_loss",
                                save_best_only=True,
                                mode="min")

early_stop   = EarlyStopping(monitor="val_loss",
                             patience=5,
                             mode="min",
                             verbose=1)

callbacks = [lr_scheduler, checkpointer, early_stop]

# ------------------------------------------------------------------
# 6. Build and train model
# ------------------------------------------------------------------
model = resnet50_unet(input_size=(TILE_SIZE, TILE_SIZE, 3),
                      freeze_encoder=True)

history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    epochs=EPOCHS,
                    validation_data=(X_val, y_val),
                    callbacks=callbacks)

# ------------------------------------------------------------------
# 7. Evaluate and save
# ------------------------------------------------------------------
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f} | Test Accuracy: {acc:.4f}")

model.save("forest_detection_resnet_unet_full.h5")

# ------------------------------------------------------------------
# 8. Plot losses
# ------------------------------------------------------------------
plt.figure(figsize=(6, 4))
plt.plot(history.history["loss"],     label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.savefig("resnet_unet_training_curve.png")
plt.close()
print("Training curve saved to resnet_unet_training_curve.png")

Processing Image: 2.TIF, Mask: 2_mask.TIF
Original Image Dimensions: (7151, 7941, 3)
Epoch 1/100
21/21 [==============================] - ETA: 0s - loss: 0.5415 - accuracy: 0.7652

C:\Users\Firoz\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


21/21 [==============================] - 154s 7s/step - loss: 0.5415 - accuracy: 0.7652 - val_loss: 2.4597 - val_accuracy: 0.7067 - lr: 1.0000e-04
Epoch 2/100
21/21 [==============================] - 156s 7s/step - loss: 0.3703 - accuracy: 0.8730 - val_loss: 1.9438 - val_accuracy: 0.8568 - lr: 1.0000e-04
Epoch 3/100
21/21 [==============================] - 159s 8s/step - loss: 0.3122 - accuracy: 0.8824 - val_loss: 1.9083 - val_accuracy: 0.8751 - lr: 1.0000e-04
Epoch 4/100
21/21 [==============================] - 155s 7s/step - loss: 0.2764 - accuracy: 0.8859 - val_loss: 1.9355 - val_accuracy: 0.8706 - lr: 1.0000e-04
Epoch 5/100
21/21 [==============================] - 155s 7s/step - loss: 0.2573 - accuracy: 0.8914 - val_loss: 1.8949 - val_accuracy: 0.8733 - lr: 1.0000e-04
Epoch 6/100
21/21 [==============================] - 154s 7s/step - loss: 0.2553 - accuracy: 0.8884 - val_loss: 1.9207 - val_accuracy: 0.8796 - lr: 1.0000e-04
Epoch 7/100
21/21 [==============================] - 154s 

In [10]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('best_resnet_unet_1572025.h5')

# Predictions on the test set
y_pred = loaded_model.predict(X_test)

# Threshold the predictions to get binary values (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Flatten the arrays for metrics calculation
y_test_flat = y_test.flatten()
y_pred_flat = y_pred_binary.flatten()

# Calculate and print accuracy
accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(f'Accuracy: {accuracy:.4f}')

# Calculate and print F1 score
f1 = f1_score(y_test_flat, y_pred_flat)
print(f'F1 Score: {f1:.4f}')

# Calculate and print precision
precision = precision_score(y_test_flat, y_pred_flat)
print(f'Precision: {precision:.4f}')

# Calculate and print recall
recall = recall_score(y_test_flat, y_pred_flat)
print(f'Recall: {recall:.4f}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_flat, y_pred_flat)
print('Confusion Matrix:')
print(conf_matrix)

# Calculate mean intersection over union (mIoU)
intersection = np.sum(np.logical_and(y_test_flat, y_pred_flat))
union = np.sum(np.logical_or(y_test_flat, y_pred_flat))
miou = intersection / union
print(f'Mean Intersection over Union (mIoU): {miou:.4f}')


6/6 [==============================] - 61s 8s/step
Accuracy: 0.8899
F1 Score: 0.8722
Precision: 0.9854
Recall: 0.7824
Confusion Matrix:
[[6061663   65549]
 [1233784 4435484]]
Mean Intersection over Union (mIoU): 0.7734


In [9]:
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    confusion_matrix,
    roc_auc_score
)
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('best_resnet_unet_1572025.h5')

# Predict probabilities on the test set
y_pred = loaded_model.predict(X_test)

# Threshold the predictions to get binary values (0 or 1)
y_pred_binary = (y_pred > 0.5).astype(int)

# Flatten arrays for metric calculations
y_test_flat = y_test.flatten()
y_pred_flat = y_pred_binary.flatten()
y_pred_prob_flat = y_pred.flatten()  # Keep raw probabilities for ROC-AUC

# Accuracy
accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(f'Accuracy: {accuracy:.4f}')

# F1 Score
f1 = f1_score(y_test_flat, y_pred_flat)
print(f'F1 Score: {f1:.4f}')

# Precision
precision = precision_score(y_test_flat, y_pred_flat)
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_test_flat, y_pred_flat)
print(f'Recall: {recall:.4f}')

# Confusion Matrix
conf_matrix = confusion_matrix(y_test_flat, y_pred_flat)
print('Confusion Matrix:')
print(conf_matrix)

# Mean Intersection over Union (mIoU)
intersection = np.sum(np.logical_and(y_test_flat, y_pred_flat))
union = np.sum(np.logical_or(y_test_flat, y_pred_flat))
miou = intersection / union
print(f'Mean Intersection over Union (mIoU): {miou:.4f}')

# ROC-AUC Score
roc_auc = roc_auc_score(y_test_flat, y_pred_prob_flat)
print(f'ROC-AUC Score: {roc_auc:.4f}')

6/6 [==============================] - 63s 8s/step
Accuracy: 0.8899
F1 Score: 0.8722
Precision: 0.9854
Recall: 0.7824
Confusion Matrix:
[[6061663   65549]
 [1233784 4435484]]
Mean Intersection over Union (mIoU): 0.7734
ROC-AUC Score: 0.8917
